In [114]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [ ]:
# Imports
import torch
from torch import nn
from torch.nn import functional as F
import matplotlib.pyplot as plt
from torchvision import models, transforms, datasets
from PIL import Image
import zipfile

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/50045 Information Retrieval/train_images (1).zip", 'r')
zip_ref.extractall("/content/drive/MyDrive/50045 Information Retrieval/train_images")
zip_ref.close()

In [ ]:
%ls

In [ ]:
class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends torchvision.datasets.ImageFolder
    """  
    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

Image.open('train_images/22c9ac783006b672e61cb704a4e44782.jpg')

In [ ]:
#preprocess images
transforms_ = transforms.Compose([
    transforms.Resize(size=[224, 224], interpolation=2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])
])

dataset = ImageFolderWithPaths('D:\\SUTD\\Term8\\50045InfoRetrieval\\50045-IR-Project', transforms_) # our custom dataset

#define dataloader
BATCH = 1
dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH)

In [ ]:
model = models.alexnet(pretrained=True) #write your code here
model.to(DEVICE)
print(model)

In [ ]:
def pooling_output(x):
	#write your code here
    for layer_name, layer in model._modules.items():
        x = layer(x)
        if layer_name == 'avgpool':
            break
    return x.view(BATCH, -1)

vec_img_list = []

In [ ]:
with torch.no_grad():
    model.eval()
    for item, (inputs, labels, paths) in enumerate(dataloader):
        print("Batch {}/{}".format(item,len(dataloader)), end='\r')
        inputs = inputs.to(DEVICE)
        img_vec = pooling_output(inputs) #write your code here
        vec_img_list += [(img_vec.cpu(), paths[0])]
        torch.cuda.empty_cache()

In [ ]:
query_image = 'train_images/21fd77a5e19e95031e1aae48006f96c7.jpg'
img = Image.open(query_image)
Image.open(query_image) # Just to visualise

In [ ]:
input_tensor = transforms_(img)
input_tensor = input_tensor.view(1, *input_tensor.shape)

In [ ]:
with torch.no_grad():
    input_tensor = input_tensor.to(DEVICE)
    query_vector = pooling_output(input_tensor)
    query_vector = query_vector.cpu()

query_vector = query_vector.reshape(1, 9216)
cos = nn.CosineSimilarity(dim=1, eps=1e-6)

In [ ]:
N = 5
#write you code here
cos_sim_dict = {}
for i in vec_img_list:
    cos_sim_dict[i[1]] = cos(query_vector, i[0])

# print(cos_sim_dict)
cos_sim_dict_sorted = {k: v for k, v in sorted(cos_sim_dict.items(), key=lambda item: item[1])}
# print(cos_sim_dict_sorted)

final_ans = list(cos_sim_dict_sorted.keys())[-N-1:-1]
print(final_ans)

In [ ]:
Image.open(final_ans[0])

In [ ]:
Image.open(final_ans[-2])

In [180]:
import pandas
import re

def MAP_score(gold, predicted):
    '''
        gold (list) - matches in gold.csv
        predicted (list) - list containing posting id of top n documents
        
        e.g. gold = ['train_129225211', 'train_2278313361']
             predicted = ['train_129225211', 'train_1203498230']
    '''
    threshold = len(gold)
    
    recall = []
    precision = []
    
    relevant_indexes = []
    
    num_relevant = 0
    current_count = 0
    
    for i in range(len(predicted)):
        current_count += 1
        if(predicted[i].all() in gold):
            relevant_indexes.append(i)
            
            num_relevant += 1
            recall.append(num_relevant/threshold)
            precision.append(num_relevant/current_count)
            
        else:
            recall.append(num_relevant/threshold)
            precision.append(num_relevant/current_count)
        
    final_score = 0
    for i in relevant_indexes:
        final_score += precision[i]
    if(len(relevant_indexes) != 0):
        final_score /= len(relevant_indexes)
    else:
        return 0
    
    return final_score


def all_MAP_scores(predicted):
    '''
        predicted (list of lists): each inner list should consist of all the posting id sorted with [most relevant, ..., least relevant]
    '''
    gold = []
    # read csv and save gold into a list of list
    df = pandas.read_csv('gold.csv')
    matches = df["matches"]

    for i in matches:
        line = re.sub("['\[\]]", '', i)
        gold.append(line.split())
        
    
    total_score = 0
    for index in range(500):
        num_relevant = len(gold[index])
        top_predicted = predicted[index][:num_relevant]
        
        total_score += MAP_score(gold[index], top_predicted)
    
    MAP = total_score/500
    
    return MAP

In [ ]:
gold = []
predicted = []

# read csv and save gold into a list of list
gold_df = pandas.read_csv('gold.csv')
train_df = pandas.read_csv('train.csv')
posting_id = gold_df["posting_id"]
image_names = train_df["image"]
for i in posting_id:
    gold.append( train_df[train_df['posting_id'] == i]['image'].values )

print(gold[0])

['0000a68812bc7e98c42888dfb1c07da0.jpg']


In [136]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Pei
[nltk_data]     Yuan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [153]:
import os


for i in range(500):
    # print()
    # print(i)
    if i % 50 == 0:
        print(i)
    abc = list(gold_df[train_df['image'] == gold[i][0]]['matches'])
    

    # print(str(abc[0]).split()[0])
    line = re.sub("['\[\]]", '', abc[0])
    # print(len(line.split(" ")))
    # print(len( list(gold_df[train_df['image'] == gold[i][0]]['matches'])))
    # N = len(gold_df[train_df['image'] == gold[i][0]]['matches'].values) + 1
    # print(N)
    # print(gold_df[train_df['image'].values == gold[i][0]]['matches'])
    N = len(line.split(" "))
    # print(f"i: {i}, N: {N}")
    cos_sim_dict = {}
    query_image = "train_images/" + gold[i][0]
    # print(query_image)
    img = Image.open(str(query_image))
    input_tensor = transforms_(img)
    input_tensor = input_tensor.view(1, *input_tensor.shape)

    with torch.no_grad():
        input_tensor = input_tensor.to(DEVICE)
        query_vector = pooling_output(input_tensor)
        query_vector = query_vector.cpu()

    query_vector = query_vector.reshape(1, 9216)
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)

    for i in vec_img_list:
        cos_sim_dict[i[1]] = cos(query_vector, i[0])

    cos_sim_dict_sorted = {k: v for k, v in sorted(cos_sim_dict.items(), key=lambda item: item[1])}

    final_ans = list(cos_sim_dict_sorted.keys())[-N-1:-1]
    # print(final_ans)
    final_list = []
    for i in final_ans:
        head, tail = os.path.split(i)
        final_list.append(tail)
    predicted.append(final_list)


0
50
100
150
200
250
300
350
400
450


In [157]:
# predicted = []
print(predicted)

[['90ff48781dfc4a683314b599be8b7f23.jpg', 'db9c6459d126b666ecf93972f878e4e6.jpg'], ['0197b998b0c6f28d544a786067d6c59d.jpg', '8cbe4bf9706bc177fd61071ef776be8c.jpg'], ['88dab14c5a0a2d3e15d9eea0becfa410.jpg', 'e427afe5f5f94edda387d144c2953f45.jpg'], ['4999893d843a83a4f06c8b316ac6045c.jpg', '2f6affb3692c42bd2be5e479e636cb6a.jpg'], ['8e2f28072a95513c1ec6fe9e4512f78c.jpg', '8b0e60baf319fa282242ab1739df10e0.jpg'], ['f712d518124c5dd5878c99adc99200de.jpg', 'b713b9df4fbb501b482f3f09e8d8bd39.jpg', '0b5b2b2b3f84721140a7e18c2e43a4ff.jpg'], ['e44ec63d3f408c46ca218f44bbd3ab7f.jpg', '7b972fd786d3a37a30d632b53cb4f23e.jpg', '93d09384206e1062512667af7c8755fa.jpg', '77978f6aab3c6e42b7a5d1d4e240195e.jpg', '69cadeca81f760c41b5e3da86a038681.jpg', '1a98bb4d97e80cd80400f1b3d704f008.jpg', 'c827ea0860cef00f880b295cf9109a16.jpg'], ['4077e2d21d9815e1ef48d4512de42373.jpg', '5e397bcfbec45600e9d648562627c79b.jpg'], ['18109415f01da4ec2869b72d42c8202c.jpg', 'db3239fab46e4ceee2147b6a7085d3d7.jpg', '247fb6f523a663c3333ef

In [181]:
print(all_MAP_scores(predicted_list))

0.2710375187478659


In [164]:
predicted_list = []


for ea_list in predicted:
    temp_list = []
    for ea_name in ea_list:
        temp_list.append( train_df[train_df["image"] == ea_name]["posting_id"].values )
    predicted_list.append(temp_list)
    # print(predicted_list)   

In [173]:
print(predicted_list[0])

[array(['train_197296533'], dtype=object), array(['train_1027959456'], dtype=object)]


In [174]:
new_list = []
for i in predicted_list:
    tmp_list=[]
    for j in i:
        tmp_list.append(j.tolist())
    new_list.append(tmp_list)